### 1. BFS路线搜索

In [15]:

BEIJING, CHANGCHUN, URUMCHI, WUHAN, GUNAGHZOU, SHENZHEN, BANGKOK, SHANGHAI, NEWYORK = """
BEIJING CHANGCHUN URUMCHI WUHAN GUANGZHOU SHENZHEN BANGKOK SHANGHAI NEWYORK
""".split()

connection_2 = {
    0: [1, 5], 
    1: [0, 2], 
    2: [1, 3], 
    3: [2, 4], 
    4: [3],
    5: [0, 6], 
    6: [5, 7],
    7: [6]
}

connection = {
	CHANGCHUN: [BEIJING],
	URUMCHI: [BEIJING], 
	BEIJING: [URUMCHI, CHANGCHUN,WUHAN, SHENZHEN, NEWYORK],
	NEWYORK: [BEIJING, SHANGHAI],
    SHANGHAI: [NEWYORK, WUHAN],
    WUHAN: [SHANGHAI, BEIJING, GUNAGHZOU],
    GUNAGHZOU: [WUHAN, BANGKOK],
    SHENZHEN: [WUHAN, BANGKOK],
    BANGKOK: [SHENZHEN, GUNAGHZOU]
}

def nagivator(start, destination, connection_graph):
	pathes = [[start]]
	seen = set()

	while pathes:
		path = pathes.pop(0)
		frontier = path[-1]

		if frontier is seen:
			continue
		seen.add(frontier)
		successors = connection_graph[frontier]

		for s in successors:
			if s == destination:
				path.append(s)
				return path
			else:
				pathes.append(path+[s])
		pathes = sorted(pathes, key = len)

	return []

print('✈️-> '.join(nagivator(GUNAGHZOU, BEIJING, connection)))

GUANGZHOU✈️-> WUHAN✈️-> BEIJING


### 2. 文本自动匹配

In [3]:
import random

grammar = """
sentence => noun_phrase verb_phrase 
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的
"""

def parse_grammar(grammar_str, sep = '=>'):
    grammar = {}
    for line in grammar_str.split('\n'):
        line = line.strip()
        if not line:
            continue

        target, rules = line.split(sep)
        grammar[target.strip()] = [r.split() for r in rules.split('|')]
    return grammar

def gene(grammar_parsed, target = 'sentence'):
    if target not in grammar_parsed:
        return target
    r = grammar_parsed[target]
    print('r',r)
    rule = random.choice(r)
    print('rele',rule)
    return ''.join(gene(grammar_parsed, target=r) for r in rule if r != 'null')


g = parse_grammar(grammar)

print(gene(g))

r [['noun_phrase', 'verb_phrase']]
rele ['noun_phrase', 'verb_phrase']
r [['Article', 'Adj*', 'noun']]
rele ['Article', 'Adj*', 'noun']
r [['一个'], ['这个']]
rele ['这个']
r [['null'], ['Adj', 'Adj*']]
rele ['null']
r [['女人'], ['篮球'], ['桌子'], ['小猫']]
rele ['女人']
r [['verb', 'noun_phrase']]
rele ['verb', 'noun_phrase']
r [['看着'], ['坐在'], ['听着'], ['看见']]
rele ['坐在']
r [['Article', 'Adj*', 'noun']]
rele ['Article', 'Adj*', 'noun']
r [['一个'], ['这个']]
rele ['这个']
r [['null'], ['Adj', 'Adj*']]
rele ['Adj', 'Adj*']
r [['蓝色的'], ['好看的'], ['小小的']]
rele ['蓝色的']
r [['null'], ['Adj', 'Adj*']]
rele ['null']
r [['女人'], ['篮球'], ['桌子'], ['小猫']]
rele ['小猫']
这个女人坐在这个蓝色的小猫


### 3.Pattern Match

In [4]:
import random

def is_variable(pat):
	return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_match(pattern, saying):
	if not pattern or not saying:
		return []
	if is_variable(pattern[0]):
		return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
	elif pattern[0] != saying[0]:
			return False
	else:
		return pat_match(pattern[1:], saying[1:])


def pat_to_dict(patterns):
	return {k:v for k,v in patterns}

def substitute(rule, parsed_rules):
	if not rule:return []
	# dict.get(key, default = None):如果key不存在则返回默认值

	return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:],parsed_rules)


defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

def get_response(saying, rules = defined_patterns):
	for pattern in rules.keys():
		pat = pat_match(pattern.split(),saying.split())
		
		if pat:
			response_pattern = random.choice(rules[pattern]) 
			responce = substitute(response_pattern.split(),pat_to_dict(pat))
			return ' '.join(responce)
	return "sorry, I don't know!"


print(get_response('I need apple'))

Image you will get apple soon


### 4. Segment Match

In [7]:
import random

def is_variable(pat):
	return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

# 判断是否为‘？*‘
def is_pattern_segment(pattern):
	return pattern.startswith('?*') and all(s.isalpha() for s in pattern[2:])

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])
    
def segment_match(pattern,saying):
	seg_pat, rest = pattern[0], pattern[1:]
	seg_pat =seg_pat.replace('?*','?')

	if not rest: 
		return (seg_pat, saying),len(saying)

	for i,token in enumerate(saying):
		if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
			print('is_match',is_match(rest[1:], saying[(i + 1):]))
			return (seg_pat, saying[:i]),i

	return (seg_pat, None),0


fail = [True,None]
def pat_match_with_seg(pattern, saying):
	if not pattern or not saying:
		return []
	pat = pattern[0]

	if is_variable(pat):
		return [pat, saying[0]] + pat_match_with_seg(pattern[1:],saying[1:])
	elif is_pattern_segment(pat):
		match, index = segment_match(pattern,saying)
		return [match] + pat_match_with_seg(pattern[1:], saying[index:])
	elif pat == saying[0]:
		return pat_match_with_seg(pattern[1:],saying[1:])
	else:
		return fail

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

def substitute(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}
def get_response(saying,rules = rules):
	for pattern in rules.keys():
		pat = pat_match_with_seg(pattern.split(),saying.split())
		if pat[-1]:
			responce_pattern = random.choice(rules[pattern])
			responce = substitute(responce_pattern.split(),pat_to_dict(pat))
			return ' '.join(responce)
	return "sorry, I don't know"
    

print(get_response('I was a beautiful and sunny girl'))


I already knew you were a beautiful and sunny girl .


### 5. 中文匹配

In [8]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
}

In [14]:
import random
import re
import jieba


def is_variable(pat):
	return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

# 判断是否为‘？*‘
def is_pattern_segment(pattern):
	return pattern.startswith('?*') and all(s.isalpha() for s in pattern[2:])

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])
    
def segment_match(pattern,saying):
	seg_pat, rest = pattern[0], pattern[1:]
	seg_pat =seg_pat.replace('?*','?')

	if not rest: 
		return (seg_pat, saying),len(saying)

	for i,token in enumerate(saying):
		if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
			return (seg_pat, saying[:i]),i

	return (seg_pat, None),0


fail = [True,None]
def pat_match_with_seg(pattern, saying):
	if not pattern or not saying:
		return []
	
	pat = pattern[0]

	if is_variable(pat):
		return [pat, saying[0]] + pat_match_with_seg(pattern[1:],saying[1:])
	elif is_pattern_segment(pat):
		match, index = segment_match(pattern,saying)
		return [match] + pat_match_with_seg(pattern[1:], saying[index:])
	elif pat == saying[0]:
		return pat_match_with_seg(pattern[1:],saying[1:])
	else:
		return fail

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

def substitute(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

def get_response(saying,rules = rule_responses):
	for pattern in rules.keys():
		pat = pat_match_with_seg(change_pattern(pattern),change_pattern(saying))
		if pat[-1]:
			responce_pattern = random.choice(rules[pattern])
			responce = substitute(change_pattern(responce_pattern),pat_to_dict(pat))
			# return ' '.join(responce)
			
			return chinese_blank(' '.join(responce)) 
	return random.choice(['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?'])

def repl(matched):
	# 不使用jieba分词
	return ' '+ matched.group() +' '
	# 使用jieba分词
	# return ' '+ ' '.join(jieba.cut(matched.group())) +' '

def change_pattern(pattern):
	# 不使用jieba分词
	pat = re.compile(r'[\u4e00-\u9fa5]')
	# 使用jieba分词
	# pat = re.compile(r'[\u4e00-\u9fa5]+')
	return re.sub(pat,repl,pattern).split()

def chinese_blank(response):
	pattern = re.compile(u"[a-zA-Z]+\s+[a-zA-Z]")
	entxt = re.findall(pattern,response)
	if (not entxt):
		response = response.replace(' ','')
	return response

print(get_response('哎，你为什么不开心'))
print(get_response('I want an apple'))
print(get_response('夸夸我呀'))

等我心情好了，我就开心
Suppose you got an apple soon
请继续


### 6. 思考问题
待增补